In [ ]:
using Flux
using CPUTime
using NeuralPDE
using GalacticOptim
using Optim
using DiffEqFlux
using Quadrature
using CUDA
using Cuba
using QuasiMonteCarlo
using ModelingToolkit #it contains @variables and parameters
import ModelingToolkit: Interval, infimum, supremum

@parameters x,y
@variables u(..)

Dxx = Differential(x)^2
Dyy = Differential(y)^2
Dy = Differential(y)
Dx = Differential(x)

D=1
ν=0.3
p0=-10;

#Dimensions of plate
a = 1
b = 1

#Neural Network Parameters
dim = 2
nodes = 16
#Learning Rates
r1 = 0.1
r2 = 0.001

In [ ]:
p = (p0/D)*sin((π*x)/a)*sin((π*y)/b)

In [ ]:
c = (1/a^2) + (1/b^2)
analytical_solution(x,y) = (p0/(π^4*D*c))*sin((π*x)/a)*sin((π*y)/b)

In [ ]:
eq = [Dxx(Dxx(u(x,y))) + Dyy(Dyy(u(x,y))) + 2*Dyy(Dxx(u(x,y))) ~ p/D]

In [ ]:
bcs = [u(0, y) ~ 0.0,
        Dxx(u(0.0,y)) ~ 0.0,
        Dxx(u(a, y)) ~ 0.0,
        u(a,y) ~ 0.0,
        Dyy(u(x,0.0)) ~ 0.0,
        Dyy(u(x, b)) ~ 0.0,
        u(x, 0.0) ~ 0.0,
        u(x, b) ~ 0.0]

In [ ]:
domains = [x ∈ Interval(0.0,a),
           y ∈ Interval(0.0,b)]

In [ ]:
chain = FastChain(FastDense(dim,nodes,Flux.σ),FastDense(nodes,nodes,Flux.σ),FastDense(nodes,1))
# Initial parameters of Neural network
initθ = DiffEqFlux.initial_params(chain) |> gpu

# Training Strategy
strategy = NeuralPDE.QuasiRandomTraining(3000; #points
                                         sampling_alg = UniformSample(),
                                         minibatch = 50)
# Discritization
discretization = NeuralPDE.PhysicsInformedNN(chain,
                                             strategy;
                                             init_params = initθ)
# Problem formulation
@named pde_system = PDESystem(eq,bcs,domains,[x,y],[u])
prob = NeuralPDE.discretize(pde_system,discretization)
# Callback function
cb = function (p,l)
    println("Current loss is: $l")
    return false
end

# Training
res = GalacticOptim.solve(prob,ADAM(r1);cb=cb,maxiters=3500)
prob = remake(prob,u0=res.minimizer)
    
for i in 1:2
    res = GalacticOptim.solve(prob,ADAM(r2);cb=cb,maxiters=2500)
    prob = remake(prob,u0=res.minimizer)
end
res = GalacticOptim.solve(prob,ADAM(0r2);cb=cb,maxiters=2500)

In [ ]:
dx = 0.05
phi = discretization.phi
xs,ys = [infimum(d.domain):dx:supremum(d.domain) for d in domains]

In [ ]:
using Plots
u_predict = reshape([first(Array(phi([x, y], res.minimizer))) for x in xs for y in ys],(length(xs),length(ys)))
u_analytical = reshape([analytical_solution(x,y) for x in xs for y in ys],(length(xs),length(ys)))

In [ ]:
p1 = plot(xs, ys, u_predict, linetype=:contourf,title = "predicted");
savefig("NeuralPDEPlate_predicted.pdf")
plot(p1)

In [ ]:
p2 = plot(xs, ys, u_analytical, linetype=:contourf,title = "analytical");
savefig("NeuralPDEPlate_analytical.pdf")
plot(p2)

In [ ]:
diff_u = abs.(u_predict .- u_analytical)

In [ ]:
p3 = plot(xs, ys, diff_u, linetype=:contourf,title = "error");
savefig("NeuralPDEPlate_error.pdf")
plot(p3)